In [10]:
import time
from copy import deepcopy


def create_sudoku_csp(grid):
    """
    grid: lista de 81 ints (0 para vazio, 1..9 preenchidos)
    Retorna um dicionário com variables, domains e neighbors.
    """
    variables = list(range(81))
    domains = {}
    neighbors = {var: set() for var in variables}

    # Domínios (1..9 para vazio, fixo para preenchido)
    for i in variables:
        if grid[i] == 0:
            domains[i] = set(range(1, 10))
        else:
            domains[i] = {grid[i]}

    def row_of(i): return i // 9
    def col_of(i): return i % 9
    def box_of(i): return (row_of(i) // 3) * 3 + (col_of(i) // 3)

    # Vizinhanças: mesma linha, mesma coluna, mesma caixa 3x3
    for i in variables:
        r, c, b = row_of(i), col_of(i), box_of(i)
        for j in variables:
            if i == j:
                continue
            if row_of(j) == r or col_of(j) == c or box_of(j) == b:
                neighbors[i].add(j)

    return {
        "variables": variables,
        "domains": domains,
        "neighbors": neighbors
    }


class SudokuCSP:
    def __init__(self, grid):
        csp = create_sudoku_csp(grid)
        self.variables = csp["variables"]
        self.domains = csp["domains"]
        self.neighbors = csp["neighbors"]

        # Métricas
        self.nodes_expanded = 0
        self.backtracks = 0

        # Atribuição inicial com os valores fixos do grid
        self.initial_assignment = {
            var: next(iter(self.domains[var]))
            for var in self.variables
            if len(self.domains[var]) == 1
        }

        # Checagem rápida: se já houver conflito nos fixos, falha cedo
        if not self._assignment_is_valid(self.initial_assignment):
            raise ValueError("Tabuleiro inicial inválido: há conflito entre valores fixos.")

    def _assignment_is_valid(self, assignment):
        for var, val in assignment.items():
            for nb in self.neighbors[var]:
                if nb in assignment and assignment[nb] == val:
                    return False
        return True

    def is_consistent(self, var, value, assignment):
        # Consistência local: nenhum vizinho já atribuído pode ter o mesmo valor
        for nb in self.neighbors[var]:
            if nb in assignment and assignment[nb] == value:
                return False
        return True

    def select_unassigned_variable_blind(self, assignment):
        # Busca cega: primeira variável não atribuída
        for v in self.variables:
            if v not in assignment:
                return v
        return None

    def mrv(self, assignment):
        """
        MRV: escolhe a variável não atribuída com MENOS valores válidos (dado o assignment atual).
        Empate: mantém a primeira encontrada (simples e suficiente).
        """
        unassigned = [v for v in self.variables if v not in assignment]

        def count_valid_values(var):
            cnt = 0
            for val in self.domains[var]:
                if self.is_consistent(var, val, assignment):
                    cnt += 1
            return cnt

        return min(unassigned, key=count_valid_values) if unassigned else None

    def forward_check(self, var, value, assignment, domains):
        """
        Forward checking:
        Remove 'value' do domínio dos vizinhos não atribuídos.
        Retorna lista de (vizinho, valor_removido) para desfazer.
        Se algum domínio ficar vazio, retorna None (falha).
        """
        removed = []
        for nb in self.neighbors[var]:
            if nb in assignment:
                continue
            if value in domains[nb]:
                domains[nb].remove(value)
                removed.append((nb, value))
                if len(domains[nb]) == 0:
                    # Falha: domínio esvaziou
                    for n, val in removed:
                        domains[n].add(val)
                    return None
        return removed

    def _backtrack(self, assignment, use_mrv, domains):
        # Condição de parada
        if len(assignment) == len(self.variables):
            return assignment

        # Seleção da variável
        var = self.mrv(assignment) if use_mrv else self.select_unassigned_variable_blind(assignment)
        if var is None:
            return assignment

        self.nodes_expanded += 1

        # Tenta valores do domínio
        for value in sorted(domains[var]):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value

                # Cópia rasa controlada do domains para manter performance
                # (vamos modificar domains e depois desfazer pelo 'removed')
                removed = self.forward_check(var, value, assignment, domains)
                if removed is not None:
                    result = self._backtrack(assignment, use_mrv, domains)
                    if result is not None:
                        return result

                    # Desfaz forward checking
                    for nb, val in removed:
                        domains[nb].add(val)

                # Desfaz atribuição
                del assignment[var]

        self.backtracks += 1
        return None

    def backtracking_search(self, use_mrv=False):
        """
        Executa busca com ou sem MRV e retorna:
        solution, time, nodes_expanded, backtracks
        """
        self.nodes_expanded = 0
        self.backtracks = 0

        start_time = time.time()

        assignment = dict(self.initial_assignment)
        # Copia profunda dos domínios para a execução (para não sujar estado entre cenários)
        domains = {k: set(v) for k, v in self.domains.items()}

        solution = self._backtrack(assignment, use_mrv, domains)

        end_time = time.time()

        return {
            "solution": solution,
            "time": end_time - start_time,
            "nodes_expanded": self.nodes_expanded,
            "backtracks": self.backtracks
        }


def format_solution(solution):
    if solution is None:
        return "Sem solução."
    lines = []
    for r in range(9):
        row = []
        for c in range(9):
            row.append(str(solution[r * 9 + c]))
        lines.append(" ".join(row))
    return "\n".join(lines)


def compare_blind_vs_mrv(grid):
    # Cenário 1: busca cega
    solver1 = SudokuCSP(grid)
    blind = solver1.backtracking_search(use_mrv=False)

    # Cenário 2: MRV
    solver2 = SudokuCSP(grid)
    mrv = solver2.backtracking_search(use_mrv=True)

    # Relatório comparativo (do jeito que professor gosta)
    report = [
        {
            "Cenário": "Busca cega",
            "Tempo Total (s)": round(blind["time"], 6),
            "Nós Expandidos": blind["nodes_expanded"],
            "Backtracks": blind["backtracks"],
            "Resolveu": blind["solution"] is not None
        },
        {
            "Cenário": "MRV",
            "Tempo Total (s)": round(mrv["time"], 6),
            "Nós Expandidos": mrv["nodes_expanded"],
            "Backtracks": mrv["backtracks"],
            "Resolveu": mrv["solution"] is not None
        }
    ]

    return blind, mrv, report


# Exemplo de uso:
if __name__ == "__main__":
    # Substitua pelo seu Sudoku (0 = vazio). Precisa ter 81 números.
    grid = [
        0,0,0, 2,6,0, 7,0,1,
        6,8,0, 0,7,0, 0,9,0,
        1,9,0, 0,0,4, 5,0,0,

        8,2,0, 1,0,0, 0,4,0,
        0,0,4, 6,0,2, 9,0,0,
        0,5,0, 0,0,3, 0,2,8,

        0,0,9, 3,0,0, 0,7,4,
        0,4,0, 0,5,0, 0,3,6,
        7,0,3, 0,1,8, 0,0,0
    ]

    blind, mrv, report = compare_blind_vs_mrv(grid)

    print("Comparação (Busca cega vs MRV):")
    for row in report:
        print(row)

    print("\nSolução (Busca cega):")
    print(format_solution(blind["solution"]))

    print("\n\nSolução (MRV):")
    print(format_solution(mrv["solution"]))

Comparação (Busca cega vs MRV):
{'Cenário': 'Busca cega', 'Tempo Total (s)': 0.000366, 'Nós Expandidos': 54, 'Backtracks': 9, 'Resolveu': True}
{'Cenário': 'MRV', 'Tempo Total (s)': 0.00631, 'Nós Expandidos': 45, 'Backtracks': 0, 'Resolveu': True}

Solução (Busca cega):
4 3 5 2 6 9 7 8 1
6 8 2 5 7 1 4 9 3
1 9 7 8 3 4 5 6 2
8 2 6 1 9 5 3 4 7
3 7 4 6 8 2 9 1 5
9 5 1 7 4 3 6 2 8
5 1 9 3 2 6 8 7 4
2 4 8 9 5 7 1 3 6
7 6 3 4 1 8 2 5 9


Solução (MRV):
4 3 5 2 6 9 7 8 1
6 8 2 5 7 1 4 9 3
1 9 7 8 3 4 5 6 2
8 2 6 1 9 5 3 4 7
3 7 4 6 8 2 9 1 5
9 5 1 7 4 3 6 2 8
5 1 9 3 2 6 8 7 4
2 4 8 9 5 7 1 3 6
7 6 3 4 1 8 2 5 9
